## データの整形

In [ ]:
!conda install -y pyarrow s3fs 
#!pip freeze > requirements.txt

In [2]:
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import pyarrow.parquet as pq
import s3fs

In [13]:
def s3_load(s3_path):
    # s3からデータを読み込む
    fs = s3fs.S3FileSystem()
    dataset = pq.ParquetDataset(s3_path, filesystem=fs)
    table = dataset.read()
    df = table.to_pandas()
    return df

def s3_load_days(s3_dir_path, start_dt, end_dt):
    # s3から日毎にデータを読み込む
    dfs = []
    dt = start_dt
    while dt <= end_dt:
        dt_str = dt.strftime('%Y-%m-%d')
        print(dt_str)
        s3_dt_path = s3_dir_path + f'dt={dt_str}'
        df = s3_load(s3_dt_path)
        dfs.append(df)
        dt += relativedelta(days=1)
    dfs = pd.concat(dfs)
    return dfs

In [1]:
s3_dir_path = f's3://{bucket}/{prifix}'
start_dt = datetime.datetime(2019, 12, 1)
end_dt = datetime.datetime(2019, 12, 31)

df = s3_load_days(s3_dir_path, start_dt, end_dt)
df.head()
X =
y = 

In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
steps = [('preprocess', OneHotEncoder(handle_unknown="ignore", sparse=False)), ('model', LogisticRegression())]
pipeline = Pipeline(steps=steps)
pipeline.fit(X, y)

## 評価

In [19]:
y_prob = pipeline.predict_proba(X)
y_prob[0:3]

array([[5.38520079e-05, 1.35594518e-03, 3.24138010e-05, ...,
        1.40758938e-04, 1.42736107e-03, 1.86491739e-04],
       [1.59489337e-04, 3.80819507e-02, 4.27922787e-05, ...,
        3.62563777e-05, 9.14806259e-05, 5.71743648e-05],
       [8.06277274e-05, 1.35010804e-03, 3.26009327e-05, ...,
        8.01564446e-05, 9.89089636e-04, 2.10797420e-04]])

In [23]:
y_pred_top3 = []
for probs in y_prob:
    top3_prob = sorted(probs,reverse=True)[0:3]
    top3_index = [int(np.where(probs ==top3_prob[j])[0]) for j in range(3)]
    y_pred_top3.append(pipeline.classes_[top3_index])

In [26]:
# top1
acc = 0
for test, top3 in zip(y_test, y_pred_top3):
    #print(test in top3)
    if test in top3[0:1]:
        acc += 1
    
acc/len(y_pred_top3)

0.0718031528794807

In [25]:
# top3
acc = 0
for test, top3 in zip(y_test, y_pred_top3):
    #print(test in top3)
    if test in top3:
        acc += 1
    
acc/len(y_pred_top3)

0.14835481085920763